# Setup

In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf

print(tf.__version__)
print(np.__version__)

1.4.0
1.16.2


# Tensor Values
The central unit of data in TensorFlow is the tensor. A tensor consists of a set of primitive values shaped into an array of any number of dimensions. A tensor's rank is its number of dimensions, while its shape is a tuple of integers specifying the array's length along each dimension. Here are some examples of tensor values:
```python
3. # a rank 0 tensor; a scalar with shape [],
[1., 2., 3.] # a rank 1 tensor; a vector with shape [3]
[[1., 2., 3.], [4., 5., 6.]] # a rank 2 tensor; a matrix with shape [2, 3]
[[[1., 2., 3.]], [[7., 8., 9.]]] # a rank 3 tensor with shape [2, 1, 3]
```
# TensorFlow Core Walkthrough
You might think of TensorFlow Core programs as consisting of two discrete sections:

1. Building the computational graph (a `tf.Graph`).
2. Running the computational graph (using a `tf.Session`).

## Graph
A computational graph is a series of TensorFlow operations arranged into a graph. The graph is composed of two types of objects.

- tf.Operation (or "ops"): The nodes of the graph. Operations describe calculations that consume and produce tensors.
- tf.Tensor: The edges in the graph. These represent the values that will flow through the graph. Most TensorFlow functions return tf.Tensors.
Important: tf.Tensors do not have values, they are just handles to elements in the computation graph.

Let's build a simple computational graph. The most basic operation is a constant. The Python function that builds the operation takes a tensor value as input. The resulting operation takes no inputs. When run, it outputs the value that was passed to the constructor. We can create two floating point constants a and b as follows:

In [3]:
a = tf.constant(3.0, dtype=tf.float32)
b = tf.constant(4.0) # also tf.float32 implicitly
total = a + b
print(a)
print(b)
print(total)

Tensor("Const:0", shape=(), dtype=float32)
Tensor("Const_1:0", shape=(), dtype=float32)
Tensor("add:0", shape=(), dtype=float32)


In [4]:
writer = tf.summary.FileWriter('.')
writer.add_graph(tf.get_default_graph())
writer.flush()

## TensorBoard
TensorFlow provides a utility called TensorBoard. One of TensorBoard's many capabilities is visualizing a computation graph. You can easily do this with a few simple commands.
First you save the computation graph to a TensorBoard summary file as follows:

This will produce an event file in the current directory with a name in the following format:
```
events.out.tfevents.{timestamp}.{hostname}
```
Now, in a new terminal, launch TensorBoard with the following shell command:

In [5]:
# ! tensorboard --logdir .

Then open TensorBoard's graphs page in your browser, and you should see a graph similar to the following:
![](https://camo.githubusercontent.com/56df99992f0a59e47d3c4e2ccee5f56c8a030654df820bcfb1144ac6e088c53d/68747470733a2f2f7777772e74656e736f72666c6f772e6f72672f696d616765732f67657474696e675f737461727465645f6164642e706e67)

# Session
To evaluate tensors, instantiate a `tf.Session` object, informally known as a session. A session encapsulates the state of the TensorFlow runtime, and runs TensorFlow operations. If a `tf.Graph` is like a `.py` file, a `tf.Session` is like the `python` executable.
The following code creates a `tf.Session` object and then invokes its `run` method to evaluate the `total` tensor we created above:

In [6]:
sess = tf.Session()
print(sess.run(total))

7.0


When you request the output of a node with `Session.run` TensorFlow backtracks through the graph and runs all the nodes that provide input to the requested output node. So this prints the expected value of 7.0.

You can pass multiple tensors to `tf.Session.run`. The `run` method transparently handles any combination of tuples or dictionaries, as in the following example:

In [7]:
print(sess.run({'ab':(a, b), 'total':total}))

{'ab': (3.0, 4.0), 'total': 7.0}


During a call to `tf.Session.run` any `tf.Tensor` only has a single value. For example, the following code calls `tf.random_uniform` to produce a `tf.Tensor` that generates a random 3-element vector (with values in [0,1)):

In [8]:
vec = tf.random_uniform(shape=(3,))
out1 = vec + 1
out2 = vec + 2
print(sess.run(vec))
print(sess.run(vec))
print(sess.run((out1, out2)))

[0.4056865  0.6606641  0.85103655]
[0.32616675 0.39491272 0.04486156]
(array([1.78516  , 1.839306 , 1.4872143], dtype=float32), array([2.78516  , 2.8393059, 2.4872143], dtype=float32))


Some TensorFlow functions return tf.Operations instead of tf.Tensors. The result of calling run on an Operation is None. You run an operation to cause a side-effect, not to retrieve a value. Examples of this include the initialization, and training ops demonstrated later.

## Feeding
As it stands, this graph is not especially interesting because it always produces a constant result. A graph can be parameterized to accept external inputs, known as placeholders. A placeholder is a promise to provide a value later, like a function argument.

In [9]:
x = tf.placeholder(tf.float32)
y = tf.placeholder(tf.float32)
z = x + y

The preceding three lines are a bit like a function in which we define two input parameters (`x` and `y`) and then an operation on them. We can evaluate this graph with multiple inputs by using the feed_dict argument of the `tf.Session.run` method to feed concrete values to the placeholders:

In [10]:
print(sess.run(z, feed_dict={x: 3, y: 4.5}))
print(sess.run(z, feed_dict={x: [1, 3], y: [2, 4]}))

7.5
[3. 7.]


Also note that the `feed_dict` argument can be used to overwrite any tensor in the graph. The only difference between placeholders and other `tf.Tensors `is that placeholders throw an error if no value is fed to them.

## Datasets
Placeholders work for simple experiments, but `tf.data` are the preferred method of streaming data into a model.

To get a runnable `tf.Tensor` from a Dataset you must first convert it to a `tf.data.Iterator`, and then call the Iterator's `tf.data.Iterator.get_next` method.

The simplest way to create an Iterator is with the `tf.data.Dataset.make_one_shot_iterator` method. For example, in the following code the `next_item` tensor will return a row from the `my_data` array on each `run` call:

In [11]:
my_data = [
    [0, 1,],
    [2, 3,],
    [4, 5,],
    [6, 7,],
]
slices = tf.data.Dataset.from_tensor_slices(my_data)
next_item = slices.make_one_shot_iterator().get_next()

Reaching the end of the data stream causes Dataset to throw an `tf.errors.OutOfRangeError`. For example, the following code reads the `next_item` until there is no more data to read:

In [12]:
while True:
  try:
    print(sess.run(next_item))
  except tf.errors.OutOfRangeError:
    break

[0 1]
[2 3]
[4 5]
[6 7]


If the `Dataset` depends on stateful operations you may need to initialize the iterator before using it, as shown below:

In [13]:
r = tf.random_normal([10,3])
dataset = tf.data.Dataset.from_tensor_slices(r)
iterator = dataset.make_initializable_iterator()
next_row = iterator.get_next()

sess.run(iterator.initializer)
while True:
  try:
    print(sess.run(next_row))
  except tf.errors.OutOfRangeError:
    break


[-1.0520592  -0.08617865  0.3474472 ]
[-0.61608595  0.4417656  -0.93556446]
[ 1.1063114  -0.22176258  0.91422236]
[ 0.04787457  0.61050564 -1.0148712 ]
[ 0.01221197 -0.07198841 -1.4249272 ]
[ 0.79151815  0.8498342  -0.96734583]
[-1.3760494  -0.4249125   0.13860041]
[ 0.7186615  -1.7870277   0.32590535]
[0.53591996 0.514571   1.0378296 ]
[0.58924747 0.9843799  0.8550042 ]


## Layers
A trainable model must modify the values in the graph to get new outputs with the same input. `tf.layers` are the preferred way to add trainable parameters to a graph.
Layers package together both the variables and the operations that act on them. For example a densely-connected layer performs a weighted sum across all inputs for each output and applies an optional activation function. The connection weights and biases are managed by the layer object.

### Creating Layers
The following code creates a `tf.layers.Dense` layer that takes a batch of input vectors, and produces a single output value for each. To apply a layer to an input, call the layer as if it were a function. For example:

In [14]:
x = tf.placeholder(tf.float32, shape=[None, 3])
linear_model = tf.layers.Dense(units=1)
y = linear_model(x)

The layer inspects its input to determine sizes for its internal variables. <mark>So here we must set the shape of the `x` placeholder so that the layer can build a weight matrix of the correct size.</mark>
Now that we have defined the calculation of the output, `y`, there is one more detail we need to take care of before we run the calculation.

### Initializing Layers
The layer contains variables that must be **initialized** before they can be used. While it is possible to initialize variables individually, you can easily initialize all the variables in a TensorFlow graph as follows:

In [15]:
init = tf.global_variables_initializer()
sess.run(init)

Important: Calling `tf.global_variables_initializer` only creates and returns a handle to a TensorFlow operation. That op will initialize all the global variables when we run it with `tf.Session.run`.
Also note that this `global_variables_initializer` only initializes variables that existed in the graph when the initializer was created. <mark>So the initializer should be one of the last things added during graph construction.</mark>

### Executing Layers
Now that the layer is initialized, we can evaluate the `linear_model`'s output tensor as we would any other tensor. For example, the following code:

In [16]:
print(sess.run(y, {x: [[1, 2, 3],[4, 5, 6]]}))

[[-3.776684]
 [-8.938715]]


### Layer Function shortcuts
For each layer class (like `tf.layers.Dense`) TensorFlow also supplies a shortcut function (like `tf.layers.dense`). The only difference is that the <mark>shortcut function versions create and run the layer in a single call.</mark> For example, the following code is equivalent to the earlier version:

In [17]:
x = tf.placeholder(tf.float32, shape=[None, 3])
y = tf.layers.dense(x, units=1)

init = tf.global_variables_initializer()
sess.run(init)

print(sess.run(y, {x: [[1, 2, 3], [4, 5, 6]]}))
sess.close()

[[1.2749186]
 [3.8400335]]


While convenient, this approach allows no access to the `tf.layers.Layer `object. This makes introspection and debugging more difficult, and layer reuse impossible.

## Feature columns
The easiest way to experiment with feature columns is using the `tf.feature_column.input_layer` function. This function only accepts dense columns as inputs, so to view the result of a categorical column you must wrap it in an `tf.feature_column.indicator_column`. For example:

In [18]:
features = {
    'sales' : [[5], [10], [8], [9]],
    'department': ['sports', 'sports', 'gardening', 'gardening']}

department_column = tf.feature_column.categorical_column_with_vocabulary_list(
        'department', ['sports', 'gardening'])
department_column = tf.feature_column.indicator_column(department_column)

columns = [
    tf.feature_column.numeric_column('sales'),
    department_column
]

inputs = tf.feature_column.input_layer(features, columns)

Running the `inputs` tensor will parse the `features` into a batch of vectors.

Feature columns can have internal state, like layers, so they often need to be initialized. Categorical columns use `tf.contrib.lookup` internally and these require a separate initialization op, `tf.tables_initializer`.

In [19]:
var_init = tf.global_variables_initializer()
table_init = tf.tables_initializer()
sess = tf.Session()
sess.run((var_init, table_init))

(None, None)

Once the internal state has been initialized you can run `inputs` like any other `tf.Tensor`:

In [20]:
print(sess.run(inputs))

[[ 1.  0.  5.]
 [ 1.  0. 10.]
 [ 0.  1.  8.]
 [ 0.  1.  9.]]


This shows how the feature columns have packed the input vectors, with the one-hot "department" as the first two indices and "sales" as the third.

[For more information](https://zhuanlan.zhihu.com/p/73701872)

## Training
Now that you're familiar with the basics of core TensorFlow, let's train a small regression model manually.

### Define the data
First let's define some inputs, `x`, and the expected output for each input, `y_true`:

In [21]:
x = tf.constant([[1], [2], [3], [4]], dtype=tf.float32)
y_true = tf.constant([[0], [-1], [-2], [-3]], dtype=tf.float32)

### Define the model
Next, build a simple linear model, with 1 output:

In [22]:
linear_model = tf.layers.Dense(units=1)

y_pred = linear_model(x)

You can evaluate the predictions as follows:

In [23]:
init = tf.global_variables_initializer()
sess.run(init)

print(sess.run(y_pred))

[[1.0994786]
 [2.1989572]
 [3.2984357]
 [4.3979144]]


### Loss
To optimize a model, you first need to define the loss. We'll use the mean square error, a standard loss for regression problems.

While you could do this manually with lower level math operations, the t`f.losses` module provides a set of common loss functions. You can use it to calculate the mean square error as follows:

In [24]:
loss = tf.losses.mean_squared_error(labels=y_true, predictions=y_pred)

print(sess.run(loss))

23.561184


### Training
TensorFlow provides **optimizers** implementing standard optimization algorithms. These are implemented as sub-classes of tf.train.Optimizer. They incrementally change each variable in order to minimize the loss. The simplest optimization algorithm is **gradient descent**, implemented by `tf.train.GradientDescentOptimizer`. It modifies each variable according to the magnitude of the derivative of loss with respect to that variable. For example:

In [25]:
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

This code builds all the graph components necessary for the optimization, and returns a training operation. When run, the training op will update variables in the graph. You might run it as follows:

In [26]:
for i in range(100):
  _, loss_value = sess.run((train, loss))
  print(loss_value)

23.561184
16.466928
11.543665
8.126814
5.755234
4.1089497
2.9659376
2.1721392
1.6206567
1.2373165
0.97065043
0.7849463
0.6554238
0.5648888
0.5014101
0.45670933
0.42504182
0.4024219
0.38608378
0.37410823
0.36516356
0.35832584
0.35295382
0.34860262
0.34496343
0.34182197
0.3390296
0.33648312
0.33411083
0.33186316
0.32970545
0.32761374
0.32557148
0.3235669
0.32159215
0.31964153
0.31771103
0.31579807
0.31390065
0.31201738
0.3101474
0.30828995
0.30644456
0.30461088
0.30278867
0.30097762
0.2991776
0.2973885
0.29561025
0.29384267
0.2920857
0.29033923
0.28860337
0.2868778
0.28516254
0.2834576
0.28176284
0.2800782
0.2784037
0.27673912
0.27508456
0.27343982
0.271805
0.27017993
0.26856452
0.26695883
0.26536277
0.26377618
0.26219916
0.26063144
0.2590732
0.25752425
0.25598451
0.25445402
0.25293273
0.25142047
0.24991725
0.24842305
0.24693778
0.24546134
0.24399377
0.242535
0.24108487
0.23964348
0.23821068
0.23678648
0.23537077
0.2339635
0.2325647
0.23117423
0.22979209
0.22841823
0.22705251
0.22569504


Since `train` is an op, not a tensor, it doesn't return a value when run. To see the progression of the loss during training, we run the loss tensor at the same time, producing output like the following:

### Complete program

```python
x = tf.constant([[1], [2], [3], [4]], dtype=tf.float32)
y_true = tf.constant([[0], [-1], [-2], [-3]], dtype=tf.float32)

linear_model = tf.layers.Dense(units=1)

y_pred = linear_model(x)
loss = tf.losses.mean_squared_error(labels=y_true, predictions=y_pred)

optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

init = tf.global_variables_initializer()

sess = tf.Session()
sess.run(init)
for i in range(100):
  _, loss_value = sess.run((train, loss))
  print(loss_value)

print(sess.run(y_pred))
```